In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7a2f46ac45494b77c962f64f99ab6eca0fe29e090faf04b493ff61b9c723d967
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
#importing pyspark
import pyspark
#importing sparksession
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("business_case").getOrCreate()

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header", "true").csv("airlines1.csv")

In [ ]:
#AirlineDF.show()

In [ ]:
AirlineDF5 = AirlineDF.select("_c0","Year","Quarter", "Month", "DayofMonth", "DayofWeek" , \
                              "FlightDate","Reporting_Airline","DOT_ID_Reporting_Airline", \
                              "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", \
                              "OriginAirportID","OriginAirportSeqID", "OriginCityMarketID", "Origin", \
                              "OriginCityName" , "OriginState", "OriginStateFips", "OriginStateName", \
                              "OriginWac","DestAirportID","DestAirportSeqID", "DestCityMarketID", "Dest", \
                              "DestCityName" , "DestState", "DestStateFips", "DestStateName", "DestWac", \
                              "CRSDepTime","DepTime", "DepDelay", "DepDelayMinutes", "DepDel15" , \
                              "DepartureDelayGroups", "DepTimeBlk", "TaxiOut", "WheelsOff","WheelsOn", \
                              "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay" , "ArrDelayMinutes", "ArrDel15", \
                              "ArrivalDelayGroups", "ArrTimeBlk","Cancelled","CancellationCode", \
                              "Diverted","CRSElapsedTime","ActualElapsedTime", "AirTime", "Flights", \
                              "Distance" , "DistanceGroup","CarrierDelay","WeatherDelay", "NASDelay", \
                              "SecurityDelay", "LateAircraftDelay" )

# Case Study 2 
On time Report for all flights, distance travel by each flight 200 miles and more than 200 miles by Origin Airport

In [ ]:
#calculate total no. of flights
df1 = AirlineDF5.select('OriginCityName').filter("Distance >= 200").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Flights").orderBy("OriginCityName")

In [ ]:
df1.show(5)

+---------------+-------------+
| OriginCityName|Total Flights|
+---------------+-------------+
|   Aberdeen, SD|            3|
|    Abilene, TX|            2|
|Adak Island, AK|            2|
|  Aguadilla, PR|            4|
|      Akron, OH|           35|
+---------------+-------------+
only showing top 5 rows



In [ ]:
# calculate total no of on time departure flights
df2 = AirlineDF5.select('OriginCityName','DepDelayMinutes') \
.filter("Distance >= 200").filter("DepDelayMinutes < 10").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total OnTime Departure Flights").orderBy("OriginCityName")

In [ ]:
df2.show()

+--------------------+------------------------------+
|      OriginCityName|Total OnTime Departure Flights|
+--------------------+------------------------------+
|        Aberdeen, SD|                             2|
|         Abilene, TX|                             2|
|     Adak Island, AK|                             1|
|       Aguadilla, PR|                             3|
|           Akron, OH|                            30|
|          Albany, NY|                            80|
|     Albuquerque, NM|                           221|
|      Alexandria, LA|                             3|
|Allentown/Bethleh...|                            33|
|        Amarillo, TX|                            34|
|       Anchorage, AK|                           111|
|        Appleton, WI|                             5|
|   Arcata/Eureka, CA|                             9|
|       Asheville, NC|                             5|
|           Aspen, CO|                             5|
|         Atlanta, GA|      

In [ ]:
#calculate total no of on time arrival flights
df3 = AirlineDF5.select('OriginCityName','ArrDelayMinutes') \
.filter("Distance >= 200").filter("ArrDelayMinutes < 10").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total OnTime Arrival Flights").orderBy("OriginCityName")

In [ ]:
df3.show()

+--------------------+----------------------------+
|      OriginCityName|Total OnTime Arrival Flights|
+--------------------+----------------------------+
|        Aberdeen, SD|                           2|
|         Abilene, TX|                           2|
|     Adak Island, AK|                           2|
|       Aguadilla, PR|                           2|
|           Akron, OH|                          26|
|          Albany, NY|                          78|
|     Albuquerque, NM|                         212|
|      Alexandria, LA|                           3|
|Allentown/Bethleh...|                          32|
|        Amarillo, TX|                          32|
|       Anchorage, AK|                          95|
|        Appleton, WI|                           4|
|   Arcata/Eureka, CA|                          10|
|       Asheville, NC|                           5|
|           Aspen, CO|                           4|
|         Atlanta, GA|                        1758|
|   Atlantic

In [ ]:
#calculate total no of cancelled flights
df4 = AirlineDF5.select('OriginCityName','Cancelled') \
.filter("Distance >= 200").filter("Cancelled  == 1").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Cancelled Flights").orderBy("OriginCityName")

In [ ]:
#calculate total no of diverted flights
df5 = AirlineDF5.select('OriginCityName','Diverted').filter("Distance >= 200").filter("Diverted  == 1").groupBy('OriginCityName') \
.count().withColumnRenamed("count", "Total Diverted Flights").orderBy("OriginCityName")

In [ ]:
#join all dataframe
OnTimeCityReport = df1.alias("df1")\
.join(df2.alias("df2"),["OriginCityName"])\
.join(df3.alias("df3"),["OriginCityName"])\
.join(df4.alias("df4"),["OriginCityName"])\
.join(df5.alias("df5"),["OriginCityName"])

In [ ]:
#complete on time flights report by origin city
OnTimeCityReport.show(3)

+----------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
|  OriginCityName|Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights|
+----------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
|     Ontario, CA|          228|                           190|                         176|                      3|                     1|
|Palm Springs, CA|           57|                            46|                          42|                      3|                     2|
|    Columbus, OH|          277|                           219|                         209|                      6|                     3|
+----------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
only showing top 3 r

In [ ]:
OnTimeCityReport.orderBy("OriginCityName").show(4)

+---------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
| OriginCityName|Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights|
+---------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
|Albuquerque, NM|          290|                           221|                         212|                      4|                     2|
|    Atlanta, GA|         2456|                          1834|                        1758|                     36|                     6|
|     Austin, TX|          213|                           173|                         163|                      1|                     1|
|  Baltimore, MD|          651|                           484|                         461|                      6|                     3|
+---------------+----------

In [ ]:
OnTimeCityReport.describe().show()

+-------+----------------+-----------------+------------------------------+----------------------------+-----------------------+----------------------+
|summary|  OriginCityName|    Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights|
+-------+----------------+-----------------+------------------------------+----------------------------+-----------------------+----------------------+
|  count|              48|               48|                            48|                          48|                     48|                    48|
|   mean|            null|          731.375|                      555.0625|           519.7708333333334|                   12.5|    2.3958333333333335|
| stddev|            null|629.3676537740251|              461.969991023641|           435.9326631382193|     13.653773541219058|    1.4838073989447285|
|    min| Albuquerque, NM|               42|                            27|             

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window

In [ ]:
# calculate result in percentage
OnTimeCityReport\
.withColumn('OnTimeDepPercent', f.col('Total OnTime Departure Flights')/f.col('Total Flights')*100)\
.withColumn('OnTimeArrPercent', f.col('Total OnTime Arrival Flights')/f.col('Total Flights')*100)\
.show(3)

+----------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+-----------------+-----------------+
|  OriginCityName|Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights| OnTimeDepPercent| OnTimeArrPercent|
+----------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+-----------------+-----------------+
|     Ontario, CA|          228|                           190|                         176|                      3|                     1|83.33333333333334|77.19298245614034|
|Palm Springs, CA|           57|                            46|                          42|                      3|                     2| 80.7017543859649|73.68421052631578|
|    Columbus, OH|          277|                           219|                         209|                      6|    

In [ ]:
from pyspark.sql.functions import col
OnTimeCityReport.orderBy(col("Total OnTime Departure Flights").desc(),col("Total OnTime Departure Flights").desc()).show()

+--------------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
|      OriginCityName|Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights|
+--------------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+
|         Chicago, IL|         2885|                          2063|                        1933|                     68|                     6|
|         Atlanta, GA|         2456|                          1834|                        1758|                     36|                     6|
|Dallas/Fort Worth...|         1990|                          1497|                        1396|                     36|                     5|
|         Houston, TX|         1511|                          1156|                        1089|                     19|                

In [ ]:
OnTimeCityReport.groupBy().max('Total OnTime Departure Flights').show()

+-----------------------------------+
|max(Total OnTime Departure Flights)|
+-----------------------------------+
|                               2063|
+-----------------------------------+



In [ ]:
from pyspark.sql import Window
import pyspark.sql.functions as psf
wT_Flights = Window.orderBy(psf.desc("Total Flights"))
testdf1 = OnTimeCityReport.withColumn(
    "Total Flights_rank", 
    psf.dense_rank().over(wT_Flights)
)

In [ ]:
# top 10 airports by numbers of on time flights
testdf1.show(10)

+--------------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+------------------+
|      OriginCityName|Total Flights|Total OnTime Departure Flights|Total OnTime Arrival Flights|Total Cancelled Flights|Total Diverted Flights|Total Flights_rank|
+--------------------+-------------+------------------------------+----------------------------+-----------------------+----------------------+------------------+
|         Chicago, IL|         2885|                          2063|                        1933|                     68|                     6|                 1|
|         Atlanta, GA|         2456|                          1834|                        1758|                     36|                     6|                 2|
|Dallas/Fort Worth...|         1990|                          1497|                        1396|                     36|                     5|                 3|
|     Los Angeles, CA|